## numpy

In [1]:
import numpy as np

In [2]:
# N은 배치 크기이며, D_in은 입력의 차원입니다.
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.

N, D_in, H, D_out = 64, 1000, 100, 10

In [3]:
# 무작위의 입력과 출력 데이터를 생성합니다.
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

In [10]:
x.shape

(64, 1000)

In [4]:
# 무작위로 가중치를 초기화합니다.
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

In [14]:
print(
    w1.shape,
    w2.shape
)

(1000, 100) (100, 10)


In [17]:
learning_rate = 1e-6
for t in range(500):
    # 순전파 단계: 예측값 y를 계산합니다.
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)
    
    # 손실(loss)을 계산하고 출력합니다.
    loss = np.square(y_pred - y).sum()
    if t%100 == 0:
        print(t, loss)
    
    # 손실에 따른 w1, w2의 변화도를 계산하고 역전파합니다.
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred) #벡터미분
    grad_h_relu = grad_y_pred.dot(w2.T) #벡터미분
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h) #벡터미분
    
    # 가중치를 갱신합니다.
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 1.0146681946648067e-08
100 3.453226572882835e-11
200 1.3123300121477219e-13
300 5.600151799602955e-16
400 2.653116518733606e-18


## PyTorch: Tensors

In [18]:
import torch

In [19]:
dtype = torch.float
device = torch.device('cuda:0')

In [27]:
# N은 배치 크기이며, D_in은 입력의 차원입니다.
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

In [28]:
# 무작위의 입력과 출력 데이터를 생성합니다.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

In [29]:
#무작위로 가중치를 초기화합니다.
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

[선형대수 연산]
1. dot : 벡터의 내적
2. mv : 행렬과 벡터의 곱
3. mm : 행력과 행렬의 곱
4. matmul : 인수의 종류에 따라서 자동으로 dot, mv, mm을 선택

[기타 연산]
1. clamp(min, max) : input 되는 값의 상한/하한을 설정하면 그 기준으로 출력된다.

In [30]:
learning_rate = 1e-6
for t in range(500):
    # 순전파 단계: 예측값 y를 계산합니다.
    h = x.mm(w1) #행렬과 행렬의 곱
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)
    
    # 손실(loss)을 계산하고 출력합니다.
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)
    
    # 손실에 따른 w1, w2의 변화도를 계싼하고 역전파합니다.
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)
    
    # 경사하강법(gradient descent)를 사용하여 가중치를 갱신합니다.
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 692.4354248046875
199 3.5412392616271973
299 0.02804386243224144
399 0.0004838769091293216
499 6.411041249521077e-05


## PyTorch: Autograd

In [31]:
import torch

In [32]:
dtype = torch.float
device = torch.device('cuda:0')

In [33]:
N, D_in, H, D_out = 64, 1000, 100, 10

In [34]:
# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
# requires_grad = False로 설정하여 역전파 중에 이 Tensor들에 대한 변화도를 계산할
# 필요가 없음을 나타냅니다.
# requires_grad의 기본값은False이다..
x = torch.randn(N, D_in, device=device, dtype=dtype, requires_grad=False)
y = torch.randn(N, D_out, device=device, dtype=dtype, requires_grad=False)

In [35]:
# 가중치를 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
# requires_grad=True로 설정하여 역전파 중에 이 Tensor들에 대한
# 변화도를 계산할 필요가 있음을 나타냅니다.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

In [36]:
learning_rate = 1e-6
for t in range(500):
    # 순전파 단계: Tensor 연산을 사용하여 예상되는 y값을 계산합니다.
    # 이는 Tensor를 사용한 순전파 단계와 완전히 동일하지만, 역전파 단계를 별도로 구현하지 않아도 되므로
    # 중간값들에 대한 참조를 갖고 있을 필요가 없습니다.
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    # Tensor 연산을 사용하여 손실을 계산하고 출력합니다.
    # loss는 (1, ) 형태의 Tensor이며, loss.item()은 loss의 스칼라 값입니다.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())
    
    # autograd를 사용하여 역전파 단계를 계산합니다.
    # 이는 requires_grad=True를 갖는 모든 Tensor에 대해 손실의 변화도를 계산합니다.
    # 이후 w1.grad와 w2.grad는 w1과 w2 각각에 대한 소실의 변화도를 갖는 Tensor가 됩니다.
    loss.backward()
    
    # 경사하강법을 사용하여 가중치를 수동으로 갱신합니다.
    # torch.no_grad()로 감싸는 이유는 가중치들이 requires_grad=True이지만
    # autograd에서는 이를 추적할 필요가 없기 때문입니다.
    # tensor.data가 tensor의 저장공간을 공유하기는 하지만, 이력을 추적하진 않습니다.
    # 또한, 이를 위해 torch.optim.SGD를 사용할 수도 있습니다.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # 가중치 갱신 후에는 수동으로 변화도를 0으로 만듭니다.
        w1.grad.zero_()
        w2.grad.zero_()

99 1229.3853759765625
199 11.426041603088379
299 0.14126911759376526
399 0.002246161224320531
499 0.00016095700266305357
